In [ ]:
import os
import rasterio
import numpy as np

count = 0
img_path = './dataset/train_img'
mask_path = './dataset/train_mask'
img_list = os.listdir(img_path)

for img_name in img_list:
    img_file = os.path.join(img_path, img_name)
    mask_name = img_name.replace('img', 'mask')
    mask_file = os.path.join(mask_path, mask_name)
    
    mask = rasterio.open(mask_file).read().transpose(1, 2, 0)
    img = rasterio.open(img_file).read().transpose(1, 2, 0)
    
    m = np.ma.masked_where(img[:, :, -2] != 0, mask[:, :, 0])
    new_mask = np.where(m.data, m.mask, 0)
    
    mask_zero_count = np.count_nonzero(mask[:, :, 0] == 0)
    new_zero_count = np.count_nonzero(new_mask == 0)
    
    if mask_zero_count != new_zero_count:
        print(img_name)
        count += 1

print(count)



In [7]:
import os
import rasterio
import csv
import pandas as pd

image_folder = './dataset/train_img/'
train_exclude_path='./dataset/train_exclude_list.csv'
imgs_list = os.listdir(image_folder)

case_dataframe = pd.read_csv(train_exclude_path)
case_exclude_list=case_dataframe['train_exclude_cases'].tolist() ##Remember 'tolist()'

print("Original image list", len(imgs_list))
print("Exclusion list", len(case_exclude_list))

def remove_common(a, b):
    a  , b = [i for i in a if i not in b], [j for j in b if j not in a]
    return a
 
img_list=remove_common(imgs_list, case_exclude_list)
print("List after exclusion",len(img_list))



Original image list 33575
Exclusion list 1177
List after exclusion 32398


In [14]:
import os
import rasterio
import csv

# Define the paths to your images
image_folder = './dataset/train_img/'
mask_folder = './dataset/train_mask/'
output_csv = './dataset/train_meta_2.csv'



def count_pixel(mask_path=''):
    with rasterio.open(mask_path) as mask_dataset:
        mask_data = mask_dataset.read(1)  # Read the first band
        mask_pixels = (mask_data == 1).sum()  
    return mask_pixels

train_num_total=10000
pixels_more_100=[]
pixels_less_100=[]


count=0
for img in img_list:
    mask = img.replace('img', 'mask')
    mask_full_path=os.path.join(mask_folder,mask)
    pixel=count_pixel(mask_path=mask_full_path)
    if pixel>=100:
        pixels_more_100.append([img,mask])
        count+=1
    else: 
        pixels_less_100.append([img,mask])
print(count)#847
pixels_less_100=pixels_less_100[:train_num_total-count]
print(len(pixels_less_100))
image_list=pixels_more_100+pixels_less_100
print(len(image_list))

train_df = pd.DataFrame(image_list, columns=['train_img', 'train_mask'])
train_df.to_csv(output_csv, index=False)
print("CSV file with image and mask lists created successfully.")


847
9153
10000
CSV file with image and mask lists created successfully.


In [15]:
case_dataframe = pd.read_csv(output_csv)
train_img_lists=case_dataframe['train_img'].tolist() 
train_mask_lists=case_dataframe['train_mask'].tolist() 

print("Length of Image: ", len(train_img_lists), train_img_lists)
print("Length of Mask: ", len(train_mask_lists), train_mask_lists)


Length of Image:  10000 ['train_img_30145.tif', 'train_img_28163.tif', 'train_img_24531.tif', 'train_img_32346.tif', 'train_img_25623.tif', 'train_img_31969.tif', 'train_img_31465.tif', 'train_img_28938.tif', 'train_img_27406.tif', 'train_img_31654.tif', 'train_img_31587.tif', 'train_img_25949.tif', 'train_img_24624.tif', 'train_img_30046.tif', 'train_img_33227.tif', 'train_img_28603.tif', 'train_img_23551.tif', 'train_img_30048.tif', 'train_img_28351.tif', 'train_img_24311.tif', 'train_img_32464.tif', 'train_img_32173.tif', 'train_img_25145.tif', 'train_img_31044.tif', 'train_img_31464.tif', 'train_img_25022.tif', 'train_img_24187.tif', 'train_img_28558.tif', 'train_img_26307.tif', 'train_img_32976.tif', 'train_img_31985.tif', 'train_img_24524.tif', 'train_img_25548.tif', 'train_img_27114.tif', 'train_img_25825.tif', 'train_img_28565.tif', 'train_img_31175.tif', 'train_img_27084.tif', 'train_img_24171.tif', 'train_img_24914.tif', 'train_img_23613.tif', 'train_img_29157.tif', 'train_im

In [ ]:
# Define the paths to your images
import random
import csv
import pandas as pd

image_folder = './data/image/train_img/'
mask_folder = './data/image/train_mask/'
output_csv = './train_img_info_wtmask.csv'
training_csv='./training_dataset.csv'

train_num_total=10000
pixels_more_100 = []
pixels_less_100=[]
count=0
with open(output_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['Mask Pixels']) >= 100:
            pixels_more_100.append(row['Filename'])
            count+=1
        else:
            pixels_less_100.append(row['Filename'])

print(count)  #889
pixels_less_100=pixels_less_100[:train_num_total-count]
print(len(pixels_less_100))
image_list=pixels_more_100+pixels_less_100
print(len(image_list))

mask_list=[]
for image in image_list:
    image_idx = image.split("_")[-1]
    mask_list.append(f'train_mask_{image_idx}')

print(len(mask_list))
print(mask_list)

df = pd.DataFrame({'train_img': image_list, 'train_mask': mask_list})

# Save the DataFrame to CSV
df.to_csv(training_csv, index=False)

print("CSV file with image and mask lists created successfully.")



'''
pixels_less_100=pixels_less_100[:]

for files in pixels_less_100:
    file_index=files.split['_'][-1]


case_indices = list(set(item[2] for item in full_dataset))

random.shuffle(case_indices)
split_ratio = 0.8
split_index = int(split_ratio * len(case_indices))

#train_img_1.tif

with open(filtered_output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(['Filenames with Mask Pixels >= 100'])
    # Write filtered entries
    for entry in filtered_entries:
        writer.writerow([entry])

print("Filtered CSV file generated successfully.")'''